#### CLONE MODEL

In [2]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 256, done.
remote: Total 256 (delta 0), reused 0 (delta 0), pack-reused 256
Receiving objects: 100% (256/256), 72.13 MiB | 41.03 MiB/s, done.
Resolving deltas: 100% (128/128), done.


In [3]:
cd first-order-model

/content/first-order-model


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### ADD FOLDER

Add folder https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing to your google drive

#### LOAD VID AND PNG

In [7]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('/content/drive/My Drive/Computer Security/PA1/first-order-motion-model/02.png')
reader = imageio.get_reader('/content/drive/My Drive/Computer Security/PA1/first-order-motion-model/04.mp4')

#filepath to directory
#source_image = imageio.imread('/content/drive/My Drive/first-order-motion-model/02.png')
#reader = imageio.get_reader('/content/drive/My Drive/first-order-motion-model/04.mp4')

#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]

fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

#### LOAD CHECKPTS

In [8]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/drive/My Drive/Computer Security/PA1/first-order-motion-model/vox-cpk.pth.tar')

#generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            #checkpoint_path='/content/drive/My Drive/first-order-motion-model/vox-cpk.pth.tar')

#### BEGIN IMG

In [9]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 211/211 [00:07<00:00, 26.59it/s]


#### RUN MY OWN DATA

In [10]:
!ffmpeg -i /content/drive/My\ Drive/Computer\ Security/PA1/first-order-motion-model/07.mkv -ss 00:00:08 -t 00:00:18 -filter:v "crop=500:650:700:50" -async 1 reporter.mp4

#!ffmpeg -i /content/drive/My\ Drive/first-order-motion-model/07.mkv -ss 00:00:08 -t 00:00:18 -filter:v "crop=500:650:700:50" -async 1 reporter.mp4


ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [14]:
source_image = imageio.imread('/content/drive/My Drive/Computer Security/PA1/first-order-motion-model/statue-02.png')
#source_image = imageio.imread('/content/drive/My Drive/first-order-motion-model/statue-02.png')

driving_video = imageio.mimread('reporter.mp4', memtest=False)


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                             adapt_movement_scale=True)

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 540/540 [00:19<00:00, 27.16it/s]


I used the 1080p 07.mkv video in the zip file first-order-model folder, and applied statue-02.png file that is also in the folder to the video. I then added two target faces: the first the reporter that does the talking and shaking of the head; and the second the guest.

As we can see, for the part with the reporter, the reporters eyes, mouth, and head movements are all captured by the model, and applied onto the statue-02. Thus the statue was showing the similar movements as the reporter.

When the guest appeared in the video, the statue exibited the same facial expressions as the guest.